In [1]:
# ============================================
# Full ipywidgets Interface (Outlook + LLM + Excel)
# End-to-end scheduling and tracker UI with NO dummy functions
# ============================================
import os
import io
import json
from typing import Dict, Any, List, Optional
from pydantic import BaseModel, Field
from PyPDF2 import PdfReader
from docx import Document
import win32com.client
import httpx
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI
from datetime import datetime
import re

import ipywidgets as widgets
from IPython.display import display, clear_output

# ============================ ENV + LLM CLIENT ============================
load_dotenv("user_env_llama.env", override=True)

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx_client
)

# ============================ OUTLOOK HELPERS ============================
def search_emails(sender=None, subject=None):
    """Fetch emails from 'your_folder_name' folder matching filters."""
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    
    # Get top-level folder (Mailbox root)
    #root_folder = outlook.Folders.Item(1)  # Or specify mailbox name: outlook.Folders["YourMailboxName"]
    root_folder = outlook.Folders["your_email"]
    
    # Get the "" folder
    try:
        app_folder = root_folder.Folders["your_desired_folder"]
    except Exception:
        raise ValueError("Could not find folder in mailbox.")
    
    messages = app_folder.Items
    messages.Sort("[ReceivedTime]", True)  # newest first

    filtered = []
    for msg in messages:
        try:
            if sender and sender.lower() not in (msg.SenderEmailAddress or "").lower():
                continue
            if subject and subject.lower() not in (msg.Subject or "").lower():
                continue
            filtered.append(msg)
        except Exception:
            continue
    
    return filtered
    
def extract_text_from_attachment(file_bytes, filename):
    if filename.lower().endswith(".pdf"):
        reader = PdfReader(io.BytesIO(file_bytes))
        return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif filename.lower().endswith(".docx"):
        doc = Document(io.BytesIO(file_bytes))
        return "\n".join([p.text for p in doc.paragraphs])
    elif filename.lower().endswith(".txt"):
        return file_bytes.decode(errors="ignore")
    else:
        return ""

def get_email_body_and_attachments(message):
    """Return email body + text from attachments."""
    all_text = message.Body or ""
    for att in message.Attachments:
        try:
            filename = att.FileName
            temp_path = os.path.join(os.getcwd(), filename)
            att.SaveAsFile(temp_path)
            with open(temp_path, "rb") as f:
                file_bytes = f.read()
            att_text = extract_text_from_attachment(file_bytes, filename)
            all_text += "\n" + att_text
            os.remove(temp_path)
        except Exception:
            continue
    return all_text

# ============================ LLM PARSERS ============================

def strip_code_fence_to_json(text: str) -> str:
    match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    return text


def parse_candidate_info(email_text):
    prompt = f"""
    Extract candidate details from the following text and return as JSON.
    Required fields:
    - candidate_name (string)
    - phone_number (string)
    - email (string)
    - notice_period (string)
    - last_working_date (string, format: YYYY-MM-DD if available)
    - current_location (string)
    - preferred_location (string)
    - total_experience (number, in years)
    - relevant_experience (number, in years)
    - certification (string)
    - current_ctc (string, in LPA or absolute)
    - expected_ctc (string, in LPA or absolute)

    Example sample for guide :
  "candidate_name": "Varun",
  "phone_number": "+91-75083xxxxx",
  "email": "vkdfglxxx@gmail.com",
  "notice_period": serving,
  "last_working_date": null,
  "current_location": null,
  "preferred_location": null,
  "total_experience": 8.92,
  "relevant_experience": 8,
  "certification": "xyz",
  "current_ctc": null,
  "expected_ctc": "17-18"

    If any field is missing or not found, set its value to null.
    Text: {email_text}
    """

    resp = client.chat.completions.create(
        model="meta/llama-3.1-8b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )
    text = resp.choices[0].message.content.strip()

    # Remove any markdown code block wrappers if present
    match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        text = match.group(1)

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        print("LLM response was not valid JSON:", text)
        return {
            "candidate_name": None,
            "phone_number": None,
            "email": None,
            "notice_period": None,
            "last_working_date": None,
            "current_location": None,
            "preferred_location": None,
            "total_experience": None,
            "relevant_experience": None,
            "certification": None,
            "current_ctc": None,
            "expected_ctc": None
        }


# ============================ STATE MODEL ============================
class ConversationState(BaseModel):
    mode: Optional[str] = None
    sender: Optional[str] = None
    subject: Optional[str] = None
    candidate_data: Dict[str, Any] = Field(default_factory=dict)
    missing_fields: List[str] = Field(default_factory=list)
    timetable: Optional[dict] = None
    panel_timings: Optional[str] = None
    slot_duration: Optional[int] = None
    candidate_name: Optional[str] = None
    candidate_email: Optional[str] = None
    candidate_pref_time: Optional[str] = None
    schedule_confirmed: bool = False
    tracker_updated: bool = False
    candidate_schedules: List[Dict[str, Any]] = Field(default_factory=list)

# ============================ EXCEL SAVE ============================

def save_candidate_to_excel(state: ConversationState, filename="candidate_tracker.xlsx"):
    data = {
        "Candidate Name": state.candidate_data.get("candidate_name"),
        "Phone Number": state.candidate_data.get("phone_number"),
        "E-Mail": state.candidate_data.get("email"),
        "Notice Period": state.candidate_data.get("notice_period"),
        "Last Working Date": state.candidate_data.get("last_working_date"),
        "Current Location": state.candidate_data.get("current_location"),
        "Preferred Location": state.candidate_data.get("preferred_location"),
        "Total Exp": state.candidate_data.get("total_experience"),
        "relevant Exp": state.candidate_data.get("relevant_experience"),
        "Certification": (
            ", ".join(state.candidate_data["certification"]) if isinstance(state.candidate_data.get("certification"), list)
            else (state.candidate_data.get("certification") or "")
        ),
        "Current CTC": state.candidate_data.get("current_ctc"),
        "Expected CTC": state.candidate_data.get("expected_ctc")
    }
    if os.path.exists(filename):
        df = pd.read_excel(filename)
        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
    else:
        df = pd.DataFrame([data])
    df.to_excel(filename, index=False)
    state.tracker_updated = True
    return state


def save_schedule_to_excel(state: ConversationState, filename="interview_schedule.xlsx"):
    records = []
    today_str = datetime.now().strftime("%Y-%m-%d")
    for sched in state.candidate_schedules:
        records.append({
            "Candidate_Name": sched.get("candidate", ""),
            "Scheduled_Time": sched.get("scheduled_time", ""),
            "Panel_Member_1": sched.get("panel_member_1", ""),
            "Panel_Member_2": sched.get("panel_member_2", ""),
            "Date": today_str
        })
    if not records:
        return state
    if os.path.exists(filename):
        existing_df = pd.read_excel(filename)
        df = pd.concat([existing_df, pd.DataFrame(records)], ignore_index=True)
    else:
        df = pd.DataFrame(records)
    df.to_excel(filename, index=False)
    return state

# ============================ MISSING INFO CHECK ============================

def calc_missing_fields(candidate_data: Dict[str, Any]) -> List[str]:
    required_fields = [
        "candidate_name", "phone_number", "email", "notice_period", "last_working_date",
        "current_location", "preferred_location", "total_experience", "relevant_experience",
        "certification", "current_ctc", "expected_ctc"
    ]
    return [f for f in required_fields if not candidate_data.get(f)]


def send_missing_details_email(to_email: str, candidate_name: str, missing_fields: List[str], your_name="Varun Gurramkonda"):
    details_list = "\n".join([f"- {field}" for field in missing_fields])
    mail_subject = "Request for Missing Details"
    mail_body = (
        f"Hi {candidate_name or ''},\n\n"
        "Could you please share the missing details mentioned below so we can proceed?\n\n"
        f"{details_list}\n\n"
        f"Thanks,\n"
        f"{your_name}"
    )
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    mail.To = to_email
    mail.Subject = mail_subject
    mail.Body = mail_body
    mail.Send()
    return {"to": to_email, "subject": mail_subject, "body": mail_body}

# ============================ LLM: PROPOSE/UPDATE TIMETABLE ============================

def propose_timetable_llm(panel_timings: str, slot_duration: int) -> dict:
    prompt = f"""
    Create an interview timetable given:
    - Panel member availability: {panel_timings}
    - Slot duration: {slot_duration} minutes

You are a scheduling assistant. 
Follow these strict rules when creating interview timetables:

1. Only use the exact availability windows provided. 
   - Do NOT invent new times.
   - Do NOT extend or shorten availability.
   - Each slot must fully fit inside one of the listed windows.
2. Each panel must have exactly 2 interviewers.
3. No interviewer can appear in more than one interview at the same time.
4. If there are not enough valid pairings, leave those slots empty instead of making conflicts.
5. Parallel interviews are allowed only if there is zero overlap in panel members.

Your job is to return only the valid timetable that satisfies these rules.
If no valid schedule can be created for a given slot, leave it blank.

    Output the timetable in **valid JSON format** with the structure:
    {{
        "timetable": [
            {{
                "slot_number": 1,
                "time": "HH:MM - HH:MM",
                "panel_member_1": "Name 1",
                "panel_member_2": "Name 2"
            }}
        ]
    }}
    Return ONLY JSON — no extra text.
    """
    resp = client.chat.completions.create(
        model="meta/llama-3.1-8b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    text = strip_code_fence_to_json(resp.choices[0].message.content.strip())
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {"timetable": []}


def update_timetable_with_change(state: ConversationState, change_req: str) -> dict:
    prompt = f"""
You are an interview timetable scheduler.

Rules:
1) 2 interviewers per slot, no overlaps for a person, parallel allowed without shared members.
2) Slot length: {state.slot_duration} minutes.
3) Use only the available panel timings:
{json.dumps(state.panel_timings, indent=2)}

Current timetable JSON:
{json.dumps(state.timetable, indent=2)}

User requested changes:
{change_req}

Update the timetable accordingly while keeping all constraints intact.
Return **valid JSON only** (same structure as before).
"""
    resp = client.chat.completions.create(
        model="meta/llama-3.1-8b-instruct",
        temperature=0.4,
        messages=[
            {"role": "system", "content": "You update interview timetables with minimal changes."},
            {"role": "user", "content": prompt}
        ],
        timeout=60
    )
    text = strip_code_fence_to_json(resp.choices[0].message.content.strip())
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return state.timetable or {"timetable": []}

# ============================ LLM: SCHEDULE CANDIDATE ============================

def propose_candidate_schedule(state: ConversationState) -> Optional[dict]:
    prompt = f"""
You are an interview scheduler.

Constraints:
1. Each interview must have exactly two interviewers.
2. Interviews can run in parallel.
3. No interviewer can be in more than one interview at the same time.
4. All slots are {state.slot_duration} minutes long.
5. Candidate {state.candidate_name} MUST be scheduled strictly within: {state.candidate_pref_time}
6. Use only the available panel timings provided earlier:
{state.panel_timings}

Current timetable:
{json.dumps(state.timetable, indent=2)}

Schedule the candidate with a suitable panel, keeping all constraints intact.
"""
    prompt += """
Return ONLY a JSON object inside triple backticks like this:
```json
{
    "candidate": "Name",
    "scheduled_time": "HH:MM-HH:MM",
    "panel_member_1": "Panel Name",
    "panel_member_2": "Panel Name"
}
No extra commentary.
"""
    resp = client.chat.completions.create(
        model="meta/llama-3.1-8b-instruct",
        temperature=0.2,
        messages=[
            {"role": "system", "content": "You assign candidates to interview slots optimally."},
            {"role": "user", "content": prompt}
        ],
        timeout=60
    )
    llm_output = resp.choices[0].message.content.strip()
    match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", llm_output, re.DOTALL)
    if not match:
        return None
    try:
        schedule_info = json.loads(match.group(1))
        return schedule_info
    except json.JSONDecodeError:
        return None

# ============================ UI WIDGETS ============================
state = ConversationState()

# ---- Tab 1: Update Tracker ----
sender_w = widgets.Text(description="Sender", placeholder="e.g. recruiter@domain.com", layout=widgets.Layout(width='50%'))
subject_w = widgets.Text(description="Subject", placeholder="Keyword(s)", layout=widgets.Layout(width='50%'))
fetch_btn = widgets.Button(description="Fetch & Parse Emails", button_style="primary")
emails_out = widgets.Output()
parsed_sel = widgets.Dropdown(description="Parsed item", options=[], layout=widgets.Layout(width='50%'))
parsed_view = widgets.Output()
save_candidate_btn = widgets.Button(description="Save Candidate to Excel", button_style="success")
missing_out = widgets.Output()
send_missing_btn = widgets.Button(description="Send Missing Details Email", button_style="warning")
update_tracker_panel = widgets.VBox([
    widgets.HBox([sender_w, subject_w, fetch_btn]),
    emails_out,
    parsed_sel,
    parsed_view,
    widgets.HBox([save_candidate_btn, send_missing_btn]),
    missing_out
])

# ---- Tab 2: Schedule Interview ----
panel_timings_w = widgets.Textarea(
    description="Panel timings", 
    placeholder="Akshay - 5 to 6\nSubbu - 12 to 1 and 2:30 to 3:30\nRanjan - 3 to 4",
    layout=widgets.Layout(width='70%', height='120px')
)
slot_duration_w = widgets.BoundedIntText(description="Slot (min)", min=10, max=180, value=45)
gen_tt_btn = widgets.Button(description="Generate Timetable (LLM)", button_style="primary")

tt_view = widgets.Output()
change_req_w = widgets.Text(description="Change request", placeholder="e.g., Nathan only 11 to 12", layout=widgets.Layout(width='70%'))
apply_change_btn = widgets.Button(description="Apply Change (LLM)", button_style="info")

cand_name_w = widgets.Text(description="Candidate Name", layout=widgets.Layout(width='40%'))
cand_email_w = widgets.Text(description="Candidate Email", layout=widgets.Layout(width='40%'))
cand_pref_w = widgets.Text(description="Preferred Time", placeholder="e.g., 11:00-11:45", layout=widgets.Layout(width='40%'))

propose_sched_btn = widgets.Button(description="Propose Schedule (LLM)", button_style="primary")
proposal_view = widgets.Output()
accept_sched_btn = widgets.Button(description="Accept & Save", button_style="success")
schedule_status_out = widgets.Output()

schedule_panel = widgets.VBox([
    widgets.HBox([panel_timings_w, slot_duration_w]),
    gen_tt_btn,
    tt_view,
    widgets.HBox([change_req_w, apply_change_btn]),
    widgets.HTML("<hr>"),
    widgets.HBox([cand_name_w, cand_email_w, cand_pref_w]),
    widgets.HBox([propose_sched_btn, accept_sched_btn]),
    proposal_view,
    schedule_status_out
])

# ---- Top-level Tabs ----
tabs = widgets.Tab(children=[update_tracker_panel, schedule_panel])
tabs.set_title(0, "Update Tracker")
tabs.set_title(1, "Schedule Interview")
display(tabs)

# ============================ TAB 1 HANDLERS ============================
_fetched_messages_cache: List[Any] = []
_parsed_candidates_cache: List[Dict[str, Any]] = []


def _refresh_parsed_dropdown():
    options = [f"{i+1}. {d.get('candidate_name') or 'Unknown'} | {d.get('email') or 'no-email'}" 
               for i, d in enumerate(_parsed_candidates_cache)]
    parsed_sel.options = list(zip(options, range(len(options))))
    parsed_sel.value = 0 if options else None


@fetch_btn.on_click
def _on_fetch_clicked(_):
    emails_out.clear_output()
    parsed_view.clear_output()
    missing_out.clear_output()
    _fetched_messages_cache.clear()
    _parsed_candidates_cache.clear()

    state.sender = sender_w.value.strip() or None
    state.subject = subject_w.value.strip() or None

    with emails_out:
        try:
            print("Searching Outlook '' folder …")
            msgs = search_emails(state.sender, state.subject)
            if not msgs:
                print("No matching emails found.")
                return
            print(f"Found {len(msgs)} email(s). Parsing with LLM…")
            for i, msg in enumerate(msgs, 1):
                text = get_email_body_and_attachments(msg)
                data = parse_candidate_info(text)
                _fetched_messages_cache.append(msg)
                _parsed_candidates_cache.append(data)
                print(f"Parsed {i}: {data.get('candidate_name') or 'Unknown'} | {data.get('email') or 'no-email'}")
            _refresh_parsed_dropdown()
            print("Done.")
        except Exception as e:
            print("Error:", e)


@parsed_sel.observe
def _on_parsed_change(change):
    if change.get('name') == 'value' and change.get('new') is not None:
        idx = change['new']
        parsed_view.clear_output()
        with parsed_view:
            data = _parsed_candidates_cache[idx]
            print(json.dumps(data, indent=2))
        state.candidate_data = data
        state.missing_fields = calc_missing_fields(data)
        missing_out.clear_output()
        with missing_out:
            if state.missing_fields:
                print("Missing fields:", state.missing_fields)
            else:
                print("All required fields present.")


@save_candidate_btn.on_click
def _on_save_candidate(_):
    if not _parsed_candidates_cache or parsed_sel.value is None:
        missing_out.clear_output()
        with missing_out:
            print("Select a parsed candidate first.")
        return
    state.candidate_data = _parsed_candidates_cache[parsed_sel.value]
    save_candidate_to_excel(state)
    missing_out.clear_output()
    with missing_out:
        print("✅ Candidate saved to candidate_tracker.xlsx")
        if state.missing_fields:
            print("Still missing:", state.missing_fields)


@send_missing_btn.on_click
def _on_send_missing(_):
    if not state.candidate_data:
        missing_out.clear_output()
        with missing_out:
            print("No candidate selected.")
        return
    #to_email = state.candidate_data.get("email") or "gvsaivarun@gmail.com"
    to_email = "gvsaivarun@gmail.com"
    info = send_missing_details_email(
        to_email=to_email,
        candidate_name=state.candidate_data.get("candidate_name") or "",
        missing_fields=state.missing_fields or calc_missing_fields(state.candidate_data)
    )
    missing_out.clear_output()
    with missing_out:
        print("📧 Missing details email sent.")
        print(json.dumps(info, indent=2))


# ============================ TAB 2 HANDLERS ============================
_current_proposal_cache: Optional[dict] = None


@gen_tt_btn.on_click
def _on_generate_tt(_):
    tt_view.clear_output()
    schedule_status_out.clear_output()
    proposal_view.clear_output()
    # read inputs
    state.panel_timings = panel_timings_w.value.strip()
    state.slot_duration = int(slot_duration_w.value)
    with tt_view:
        try:
            timetable = propose_timetable_llm(state.panel_timings, state.slot_duration)
            state.timetable = timetable
            print("Proposed timetable:\n")
            print(json.dumps(state.timetable, indent=2))
        except Exception as e:
            print("Error:", e)


@apply_change_btn.on_click
def _on_apply_change(_):
    tt_view.clear_output()
    schedule_status_out.clear_output()
    if not state.timetable:
        with tt_view:
            print("Generate a timetable first.")
        return
    change_req = change_req_w.value.strip()
    if not change_req:
        with tt_view:
            print("Enter a change request first.")
        return
    with tt_view:
        try:
            updated = update_timetable_with_change(state, change_req)
            state.timetable = updated
            print("Updated timetable:\n")
            print(json.dumps(state.timetable, indent=2))
        except Exception as e:
            print("Error:", e)


@propose_sched_btn.on_click
def _on_propose_schedule(_):
    proposal_view.clear_output()
    schedule_status_out.clear_output()
    # capture candidate inputs
    state.candidate_name = cand_name_w.value.strip() or None
    state.candidate_email = cand_email_w.value.strip() or None
    state.candidate_pref_time = cand_pref_w.value.strip() or None

    with proposal_view:
        if not (state.timetable and state.panel_timings and state.slot_duration and state.candidate_name and state.candidate_pref_time):
            print("Please make sure timetable is generated and candidate name & preferred time are filled.")
            return
        try:
            sched = propose_candidate_schedule(state)
            if not sched:
                print("LLM did not return a valid JSON schedule proposal.")
                return
            global _current_proposal_cache
            _current_proposal_cache = sched
            print("Proposed schedule:\n")
            print(json.dumps(sched, indent=2))
        except Exception as e:
            print("Error:", e)


@accept_sched_btn.on_click
def _on_accept_schedule(_):
    schedule_status_out.clear_output()
    with schedule_status_out:
        global _current_proposal_cache
        if not _current_proposal_cache:
            print("No schedule proposal to accept.")
            return
        # save
        state.candidate_schedules.append(_current_proposal_cache)
        save_schedule_to_excel(state)
        state.schedule_confirmed = True
        print("✅ Schedule accepted and saved to interview_schedule.xlsx")
        snapshot = {
            "candidate": _current_proposal_cache.get("candidate"),
            "scheduled_time": _current_proposal_cache.get("scheduled_time"),
            "panel_member_1": _current_proposal_cache.get("panel_member_1"),
            "panel_member_2": _current_proposal_cache.get("panel_member_2"),
        }
        print(json.dumps(snapshot, indent=2))
